# Advanced Python OOP: Mastering the Data Model

This notebook goes beyond the four pillars of OOP (Encapsulation, Inheritance, Polymorphism, and Abstraction) to explore the powerful features of Python's data model.

By mastering these concepts, you can create classes that are more efficient, intuitive, and well-integrated with Python's built-in functionalities.

## 1. A Deeper Look at Abstraction with ABCs

Abstract Base Classes (ABCs) can enforce not just abstract *methods*, but also abstract *properties*. This is crucial for designing robust frameworks where you need to guarantee that certain attributes will be present on all subclasses.

In [1]:
from abc import ABC, abstractmethod, abstractproperty

class MediaAsset(ABC):
    @abstractmethod
    def play(self):
        "All media assets must have a way to be played."
        pass

    @property
    @abstractmethod
    def file_format(self):
        "All media assets must declare their file format."
        pass

class Video(MediaAsset):
    def __init__(self, title):
        self.title = title

    def play(self):
        print(f"Playing video: {self.title}")

    @property
    def file_format(self):
        return "mp4"

class Audio(MediaAsset):
    def __init__(self, title):
        self.title = title

    def play(self):
        print(f"Playing audio: {self.title}")

    @property
    def file_format(self):
        return "mp3"

video = Video("The Matrix")
audio = Audio("Bohemian Rhapsody")

video.play()
print(f"Video format: {video.file_format}")

audio.play()
print(f"Audio format: {audio.file_format}")

Playing video: The Matrix
Video format: mp4
Playing audio: Bohemian Rhapsody
Audio format: mp3


## 2. Essential OOP Decorators

Decorators are functions that modify or enhance other functions. In OOP, several decorators are essential for writing clean and effective code.

### `@classmethod`

A class method receives the class itself as the first argument, not the instance. They are commonly used for factory methods, which provide alternative ways to create instances of your class.

In [2]:
import datetime

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def from_birth_year(cls, name, birth_year):
        "Creates a Person instance from a birth year."
        current_year = datetime.date.today().year
        age = current_year - birth_year
        return cls(name, age) # cls() is equivalent to Person()

    def display(self):
        print(f"{self.name} is {self.age} years old.")

# Regular instantiation
person1 = Person("John", 30)
person1.display()

# Alternative instantiation using the class method factory
person2 = Person.from_birth_year("Jane", 1995)
person2.display()

John is 30 years old.
Jane is 30 years old.


### `@staticmethod`

A static method doesn't receive the instance or the class as the first argument. It's essentially a regular function namespaced within the class. Use it for utility functions that are logically related to the class but don't need access to instance or class state.

In [3]:
class MathUtils:
    @staticmethod
    def add(a, b):
        return a + b

    @staticmethod
    def is_even(n):
        return n % 2 == 0

print(f"2 + 3 = {MathUtils.add(2, 3)}")
print(f"Is 7 even? {MathUtils.is_even(7)}")

2 + 3 = 5
Is 7 even? False


### `@property`

The `@property` decorator is a Pythonic way to create managed attributes. It lets you turn a class method into a read-only attribute, and then you can optionally define a `.setter` and `.deleter` for it. This is perfect for validation or computed properties.

In [4]:
class Employee:
    def __init__(self, first_name, last_name, salary):
        self.first_name = first_name
        self.last_name = last_name
        self._salary = salary # Convention: _salary is a protected attribute

    @property
    def email(self):
        "This is a computed property."
        return f"{self.first_name}.{self.last_name}@company.com".lower()

    @property
    def salary(self):
        "This is the 'getter' for salary."
        return self._salary

    @salary.setter
    def salary(self, value):
        "This is the 'setter' for salary, with validation."
        if value < 0:
            raise ValueError("Salary cannot be negative.")
        self._salary = value

emp = Employee("Michael", "Scott", 50000)

# Access email like an attribute, not a method
print(emp.email)

# The setter is called automatically on assignment
emp.salary = 55000
print(f"New Salary: ${emp.salary}")

# The setter will raise an error if validation fails
try:
    emp.salary = -100
except ValueError as e:
    print(f"Error: {e}")

michael.scott@company.com
New Salary: $55000
Error: Salary cannot be negative.


## 3. Mastering Special (Dunder) Methods

Special methods, often called "dunder" (double underscore) methods, allow your objects to integrate with Python's core syntax. Implementing them is key to creating intuitive, Pythonic classes.

### String Representation: `__str__` vs `__repr__`

- `__str__`: For creating a user-friendly, readable output. Called by `print()` and `str()`.
- `__repr__`: For creating an unambiguous, developer-friendly output that should, ideally, be able to recreate the object. Called by `repr()` and the interactive console.

In [5]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    def __str__(self):
        return f"'{{self.title}}' by {{self.author}}"

    def __repr__(self):
        return f"Book(title='{{self.title}}', author='{{self.author}}')"

book = Book("Dune", "Frank Herbert")
print(str(book))   # Calls __str__
print(repr(book))  # Calls __repr__
print(book)        # print() prefers __str__

'{self.title}' by {self.author}
Book(title='{self.title}', author='{self.author}')
'{self.title}' by {self.author}


### Operator Overloading: Emulating Numeric Types

You can define how standard operators like `+`, `-`, `*`, and `/` work with your objects.

In [6]:
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        "Defines the '+' operator for Vector2D objects."
        return Vector2D(self.x + other.x, self.y + other.y)

    def __mul__(self, scalar):
        "Defines the '*' operator for scalar multiplication."
        return Vector2D(self.x * scalar, self.y * scalar)

    def __repr__(self):
        return f"Vector2D({self.x}, {self.y})"

v1 = Vector2D(2, 3)
v2 = Vector2D(4, 5)

v3 = v1 + v2  # Calls v1.__add__(v2)
print(f"v1 + v2 = {v3}")

v4 = v3 * 3   # Calls v3.__mul__(3)
print(f"v3 * 3 = {v4}")

v1 + v2 = Vector2D(6, 8)
v3 * 3 = Vector2D(18, 24)


### Context Management: `__enter__` and `__exit__`

These methods allow your object to work with the `with` statement, which is perfect for managing resources like file handles or database connections, ensuring that cleanup code always runs.

In [7]:
import time

class Timer:
    def __enter__(self):
        "Called when entering the 'with' block."
        self.start_time = time.time()
        return self # The object returned here is assigned to the 'as' variable

    def __exit__(self, exc_type, exc_val, exc_tb):
        "Called when exiting the 'with' block."
        self.end_time = time.time()
        self.elapsed = self.end_time - self.start_time
        print(f"Block executed in {self.elapsed:.4f} seconds.")
        # Return False (or nothing) to propagate exceptions, True to suppress them.
        return False

with Timer() as t:
    print("Doing some work...")
    time.sleep(1)
    print("Work done.")

print("Timer object is still available after the block.")

Doing some work...
Work done.
Block executed in 1.0012 seconds.
Timer object is still available after the block.


## 4. Advanced Design Principles & Features

### Composition over Inheritance

This is a design principle that favors building complex objects by *composing* them from smaller, simpler objects rather than inheriting from complex classes. It leads to more flexible and maintainable code.

Instead of saying a `Car` **is a type of** `Engine`, you say a `Car` **has an** `Engine`.

In [8]:
class Engine:
    def start(self):
        print("Engine starting... Vroom!")

class Wheels:
    def __init__(self, number):
        self.number = number

    def rotate(self):
        print(f"{self.number} wheels are rotating.")

class Car:
    def __init__(self, make, model):
        self.make = make
        self.model = model
        # A Car HAS an Engine (Composition)
        self.engine = Engine()
        # A Car HAS Wheels (Composition)
        self.wheels = Wheels(4)

    def drive(self):
        print(f"Driving the {self.make} {self.model}.")
        self.engine.start()
        self.wheels.rotate()

my_car = Car("Honda", "Civic")
my_car.drive()

Driving the Honda Civic.
Engine starting... Vroom!
4 wheels are rotating.


### Data Classes (`@dataclass`)

Introduced in Python 3.7, data classes are a way to automatically add special methods like `__init__()`, `__repr__()`, and `__eq__()` to your classes, saving a lot of boilerplate code.

In [9]:
from dataclasses import dataclass

@dataclass(frozen=True) # frozen=True makes instances immutable
class Point:
    x: float
    y: float

p1 = Point(1.5, 2.0)
p2 = Point(1.5, 2.0)

# __repr__ is automatically generated
print(p1)

# __eq__ is automatically generated
print(f"p1 == p2? {p1 == p2}")

# Since it's frozen, trying to change it raises an error
try:
    p1.x = 3.0
except Exception as e:
    print(f"Error: {e}")

Point(x=1.5, y=2.0)
p1 == p2? True
Error: cannot assign to field 'x'


### `__slots__`

`__slots__` is a memory optimization tool. By default, Python uses a dictionary (`__dict__`) to store an object's attributes. If you have millions of instances, this can be memory-intensive. `__slots__` tells Python not to use a `__dict__`, and to only allocate space for a fixed set of attributes.

In [10]:
class Pixel:
    # By defining __slots__, we tell Python to use a more compact data structure
    # instead of a memory-hungry __dict__ for each instance.
    __slots__ = ["r", "g", "b"]

    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

p = Pixel(255, 128, 64)
print(f"Red channel: {p.r}")

# The main trade-off: you cannot add new attributes to the instance.
try:
    p.alpha = 1.0 # This would work if we didn't use __slots__
except AttributeError as e:
    print(f"Error: {e}")

Red channel: 255
Error: 'Pixel' object has no attribute 'alpha' and no __dict__ for setting new attributes
